In [1]:
import numpy as np
import pandas as pd

import sys

from os.path import expanduser
sys.path.append(expanduser('~') + '/Lab/Utils/Python/')

from Conversions.translate import *
from Strings.is_a import *

# Initial setup

In [3]:
paper_pmid = 24262822
paper_name = 'mattiazziusaj_petrovic_2014' 

In [4]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [5]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [9]:
original_data = pd.read_csv('raw_data/MattiazziUsaj_Chemosphere_2014.csv', sep=',')

In [10]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4308 x 11


In [11]:
original_data.head()

,Plate,Row,Column,ORF,Name,Thiamethoxam,Actara,Acetamiprid,Mospilan,DMSO+Pyrrolidone,Confidor
0,1,2,2,YBR030W,NaN,0.214,-0.010,-0.024,-0.282,-0.025,-0.276
1,1,2,3,YBR138C,HDR1,0.034,0.414,-0.585,-0.095,0.225,-0.227
2,1,2,4,YBR028C,NaN,0.346,0.021,0.216,-1.066,0.025,-0.889
3,1,2,5,YBR137W,NaN,0.314,0.183,0.135,-1.154,0.216,0.317
4,1,2,6,YBR027C,NaN,-0.010,-0.189,-0.141,-0.916,0.093,-0.825


In [12]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [13]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [14]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [15]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Plate, Row, Column, ORF, Name, Thiamethoxam, Actara, Acetamiprid, Mospilan, DMSO+Pyrrolidone, Confidor ]
Index: []


In [26]:
data = original_data.loc[:,['ORF','Thiamethoxam','Actara','Acetamiprid','Mospilan','DMSO+Pyrrolidone','Confidor ']]

In [27]:
data.set_index('ORF',inplace=True)

# Prepare the final dataset

In [28]:
dataset_ids = [16502,16506,16503,16505,16504,16507]

In [29]:
datasets = datasets.reindex(index=dataset_ids)

In [30]:
data.columns = datasets['name'].values

In [31]:
data = data.groupby(data.index).mean()

In [32]:
# Create row index
data.index.name='orf'

In [33]:
print('Final data dimensions: %d x %d' % (data.shape))

Final data dimensions: 4267 x 6


# Print out

In [35]:
data.to_csv(paper_name + '.txt', sep='\t')

# Save to DB

In [36]:
from IO.save_data_to_db2 import *

In [37]:
# Create column index
lst = [datasets.index.values, datasets['name'].values]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','dataset_name'])
data.columns = idx

In [38]:
save_data_to_db(data, paper_pmid)